# Fit function

based off jeremy howards great tutorial! https://pytorch.org/tutorials/beginner/nn_tutorial.html

## Setup

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from matplotlib import pyplot as plt
import numpy as np

from pathlib import Path
import requests
import pickle
import gzip
import torch

In [4]:
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"
FILENAME = "mnist.pkl.gz"

In [5]:
def unzip_data():
    with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    
    return x_train, y_train, x_valid, y_valid

## PyTorch Datasets

abstract class where you need to implement

__len__ & __getitem__ methods 

In [6]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [7]:
class MnistDataset(Dataset):
    def __init__(self, is_valid=False):
        x_train, y_train, x_valid, y_valid = unzip_data()
        
        self.x = x_train if is_valid else x_valid
        self.y = y_train if is_valid else y_valid
        self.len = self.x.shape[0] 
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.len

In [8]:
train_ds = MnistDataset()
valid_ds = MnistDataset(is_valid=True)

## PyTorch Dataloader

manages getting batches for our training loop from our dataloader

returns a python interator

In [9]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [10]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

def to_device(x, y):
    return x.to(dev), y.to(dev)

def get_data(bs):
    train_ds = MnistDataset()
    valid_ds = MnistDataset(is_valid=True)

    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
    
    train_dl = WrappedDataLoader(train_dl, to_device)
    valid_dl = WrappedDataLoader(valid_dl, to_device)
    
    return train_dl, valid_dl

In [11]:
from torch import nn
import torch.nn.functional as F

loss_func = F.cross_entropy

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

class Mnist_NN(nn.Module):
    def __init__(self, n_h):
        super().__init__()
        self.inp = nn.Linear(784, n_h)
        self.hid = nn.Linear(n_h, n_h)
        self.out = nn.Linear(n_h, 10)
        
    def forward(self, xb):
        ab1 = F.relu(self.inp(xb))
        ab2 = F.relu(self.hid(ab1))
        ab3 = F.relu(self.out(ab2))
        return ab3

In [12]:
from torch import optim

def get_model():
    model = Mnist_NN(100)
    opt = optim.SGD(model.parameters(), lr=0.02)
    return model, opt

## Write loss_batch and fit functions

<a id="q_1" >

In [13]:
to4dec = lambda a: np.around(a, decimals=4) if not a==None else None

def print_epoch_progress(epoch, train_loss=None, valid_loss=None, metrics=[]):
    metrics = [f"{key}: {to4dec(value)}" for key, value in metrics.items()]
    print(epoch, 'train loss: ', to4dec(train_loss),'valid loss: ', to4dec(valid_loss,), ' '.join(metrics))

In [14]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    
    # if validation dont update weights (you cant learn from them)

    return 

In [15]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        
        model.train() # training
        
        model.eval() # validation

        
        print_epoch_progress(epoch, training_loss, validation_loss, {'accuracy': epoch_accuracy})

In [16]:
model, opt = get_model()
train_dl, valid_dl = get_data(64)
fit(10, model, loss_func, opt, train_dl, valid_dl)

NameError: name 'training_loss' is not defined

# Learn object

In [90]:
to4dec = lambda a: np.around(a, decimals=4) if not a==None else None

listNumpy = lambda l: [x.item() for x in l]

sumLoss = lambda loss_list: np.sum(listNumpy(loss_list)) / len(loss_list)

caculate_metric = lambda metric: lambda predb, yb, : [ metric(predb, yb).item() for predb, yb in zip(predb, yb)]


def print_epoch_progress(epoch, train_loss=None, valid_loss=None, metrics=[]):
    metrics = [f"{key}: {to4dec(value)}" for key, value in metrics.items()]
    print(epoch, 'train loss: ', to4dec(train_loss),'valid loss: ', to4dec(valid_loss,), ' '.join(metrics))

class Learner(object):
    """
        Learner object holds model, optimizer and dataloaders
    """
    def __init__(self, model, opt, loss_fn, dls={}, metrics={}):
        self.model = model
        self.opt = opt
        self.loss_fn = loss_fn
        # assumes { trn: train_dl, val: valid_dl)
        self.data = dls
        self.metrics = metrics
#         self.schedule
        
        
    def descend_(self, xb, yb, is_valid=False):
        
        predb = self.model(xb)
        loss = self.loss_fn(predb, yb)

        if is_valid is not True:
            loss.backward()
            self.opt.step()
            self.opt.zero_grad()
            
        return loss, predb, yb,xb
        
    
    def fit_(self, epochs=1):
        for epoch in range(epochs):
            
             # training
            model.train()
            trn_losses,_,_,_ = zip(
                *[self.descend_(xb, yb) for xb, yb in self.data['trn'] ]
            )
            
            # validation
            model.eval()
            with torch.no_grad():
                val_losses,predb,yb,xb = zip(
                    *[self.descend_(xb, yb, is_valid=True) for xb, yb in self.data['val']]
                )
            
            training_loss = sumLoss(trn_losses)
            validation_loss = sumLoss(val_losses)
            
            
            metrics = { 
                name:np.mean(caculate_metric(fn)(predb,yb)) for name, fn in self.metrics.items()
            }
#             = [ m for predb, yb in zip(predb, yb)]
    
#             print_epoch_progress(epoch+1, training_loss, validation_loss, {'accuracy': epoch_accuracy}) 
            print(epoch+1, training_loss, validation_loss, metrics)

In [86]:
lm(1)(2)

3

In [87]:
model = Mnist_NN(1000)
opt = optim.SGD(model.parameters(), lr=0.02)
train_dl, valid_dl = get_data(64)
dls = { 'trn':train_dl, 'val':valid_dl }

learn = Learner(model, opt, F.cross_entropy, dls, {'accuracy': accuracy})

In [88]:
learn.fit_(10)

1 2.220469294080309 2.093552371120209 {'accuracy': 0.42463235294117646}
2 1.7910123326975829 1.4995104431191368 {'accuracy': 0.624016943794992}
3 1.2479599885120514 1.1474148106696966 {'accuracy': 0.6971467391914113}
4 1.0093772737843216 1.0026936872535959 {'accuracy': 0.710318094629156}
5 0.8966639584796444 0.9269135353510337 {'accuracy': 0.7207600703019925}
6 0.833601114666386 0.87666426625703 {'accuracy': 0.7262028452685422}
7 0.7932254605612178 0.8517171771020231 {'accuracy': 0.7380474744855291}
8 0.7643834821357849 0.8273643657679448 {'accuracy': 0.7493086637133528}
9 0.743919352816928 0.8115806161900005 {'accuracy': 0.7437140345573425}
10 0.7282942913140461 0.8014064652230733 {'accuracy': 0.7574128836317136}


# L-Layer Model

In [160]:
class DeepNN(nn.Module):
    ### cloze {
    def __init__(self, layers):
        super().__init__()
        print(layers)
        self.layers = {}
        for i in range(len(layers) -1):
            print(i, [layers[i], layers[i+1]])
            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])
    def forward(self, xb):
        ab = xb
        for i in range(len(layers) -1):
            ab = F.relu(self.layers[f"lin_{i}"](ab))
        return ab
    ### } cloze 

In [161]:
layers = np.array([784, 784*4, 784*8, 784*4, 784, 784/4, 784/8, 10]).astype(int)
print(layers)



model = DeepNN(layers)
learn = Learner(model, opt, F.cross_entropy, dls, {'accuracy': accuracy})

[ 784 3136 6272 3136  784  196   98   10]
[ 784 3136 6272 3136  784  196   98   10]
0 [784, 3136]
1 [3136, 6272]
2 [6272, 3136]
3 [3136, 784]
4 [784, 196]
5 [196, 98]
6 [98, 10]


In [158]:
model.layers

{'lin_0': Linear(in_features=784, out_features=3136, bias=True),
 'lin_1': Linear(in_features=3136, out_features=6272, bias=True),
 'lin_2': Linear(in_features=6272, out_features=3136, bias=True),
 'lin_3': Linear(in_features=3136, out_features=784, bias=True),
 'lin_4': Linear(in_features=784, out_features=196, bias=True),
 'lin_5': Linear(in_features=196, out_features=98, bias=True),
 'lin_6': Linear(in_features=98, out_features=10, bias=True)}

In [159]:
learn.fit_()

1 2.3031773810174054 2.303412474024936 {'accuracy': 0.09720668158567775}
